# Creating hardware in Python using CyriteHDL

CyriteHDL or short CyHDL is a **hardware definition toolkit** based on a kernel using an internal representation python notation to generate hardware or code procedurally. It is featured by an emulation layer to provide some (not full) language compatibility to the MyHDL dialect. It is assumed that you have some knowledge of Verilog or VHDL.

If you are aiming at a fresh start with a Python based HDL framework, this howto should get you going with the following basics:

0. Need to know [inference basics](basics.ipynb)
1. The (in)famous [blinky](blinky.ipynb)
   1. Designing a counter, state machine and configuration register
   1. Testing and simulating its function
2. Design methods:
   1. [Contexts, modules, functions](methods.ipynb)   
       * RTL functions: context sensitive functional descriptions
   2. [Simulation and verification](sim_intro.ipynb)
3. Synthesis
   1. [Direct synthesis to yosys](rtlil.ipynb)
   1. Synthesis via vendor tools
4. Compiling a design for a FPGA target ( **OMITTED** )
   1. Creating the board supply package
   2. Precompiled board supply packages:
      * ECP5 Versa development kit
      * ECP5 EVDK
      * Gatemate evaluation kit (planned)
   
Advanced:
* [API reference](#The-Cyrite-API-reference)

## The Cyrite API reference

**Under construction. Please refer to the MyIRL internals for the time being**

One important note: The Cyrite HDL is split in several layers:

* Cyrite library layer: top level HDL syntax with partial MyHDL compatibility
* myirl 'intermediate representation layer' kernel layer: Procedural, generator style syntax and library
  

When dealing with development of new extensions, one is mostly operating in the `myirl` domain. Thus, `myirl.library` elements mostly use IRL (intermediate representation layer) notation.
Some elements are only documented in the IRL domain, however can be used from the cyrite layer. This is yet not fully coverage-tested.

* [Signals, wires and operations](operations.ipynb)
* [Generator basics](generators.ipynb) - When to use `yield` and when not
* [Signal types and Interfaces](signals_interfaces.ipynb)
  * [Port classes](ports.ipynb)
  * [Tristate and undefined signals](stdlogic.ipynb)
* [Library concepts: Organizing your code](library.ipynb)
    * [Blackboxes, external HDL modules](library.ipynb#External-HDL-libraries)
    * Inference and design modules
* Cyrite/MyIRL Library and extensions
    * [MyIRL library reference](../../myirl/library/index.ipynb)
    * [Register concepts](../notebooks/registers.ipynb)
         * Partially assigneable signals [[myirl kernel ref]](../notebooks/member_assignment.ipynb)
         * Address decoders, [Busses](../notebooks/busses.ipynb)
         * Memory mapped range registers
* Simulation:
    * [Simulation examples](simulation.ipynb) and scenarios, waveform displays
    * [Simulation constructs](../notebooks/simulation.ipynb)
       * [Timing delays, delayed signals](../notebooks/simulation.ipynb#Delay-modelling)
    * [Co-Simulation](../notebooks/simulation.ipynb#Co-Simulation-abstraction): Driving virtual hardware via CXXRTL
    * [Simulation commands reference](../notebooks/simulation.ipynb#Simulation-command-reference)